# How to deal with timestamps

## Install

In [1]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [2]:
import syft as sy
sy.requires(SYFT_VERSION)

✅ The installed version of syft==0.8.2b2 matches the requirement >=0.8.2b0 and the requirement <0.9


In [3]:
node = sy.orchestra.launch(name="pandas-test-domain-8", port=9088, reset=True)

Starting pandas-test-domain-8 server on 0.0.0.0:9088


Waiting for server to start Done.


# Data owner: upload dataset

In [4]:
root_domain_client = node.login(email="info@openmined.org", password="changethis")

Logged into pandas-test-domain-8 as <info@openmined.org>


## Load data

It's not obvious how to deal with Unix timestamps in pandas -- it took me quite a while to figure this out. The file we're using here is a popularity-contest file I found on my system at /var/log/popularity-contest.

In [5]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from syft.service.project.project import Project
from syft.util.util import autocache, PANDAS_DATA

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 3)
plt.rcParams['font.family'] = 'sans-serif'

In [6]:
popcon = pd.read_csv(autocache(f"{PANDAS_DATA}/popularity-contest"), sep=' ', )[:-1]

In [7]:
assert len(popcon) == 2897

In [8]:
popcon.columns = ['atime', 'ctime', 'package-name', 'mru-program', 'tag']

In [9]:
set(popcon["tag"])

{'<OLD>', '<RECENT-CTIME>', nan}

## Create mock data

Lets create a mock dataset

In [10]:
# weather_types = set(weather_2012_final.Weather.to_list())

In [11]:
from random import randint
import random

In [12]:
def get_random_timestamp():
    return "135" + "".join([str(randint(0,9)) for i in range(6)])

In [13]:
def get_mock_date(i):
    return str(parse('Jun 1 2010') + timedelta(days=i))

def get_mock_row(i):
    res = dict()
    for k, function in mock_functions.items():
        res[k] = function()
    return res
    

In [14]:
mock_functions = {'atime': lambda: get_random_timestamp(),
                  'ctime': lambda: get_random_timestamp(),
                  'package-name': lambda: random.choice([
                      'libghc-stm-dev',
                      'libqtdee-dev',
                      'geoclue-ubuntu-geoip',
                      'libdesktop-agnostic0',
                      'ubuntu-extras-keyring',
                      'libbsd0',
                      'libxres-dev']),
                  'mru-program': lambda: random.choice([
                      '/usr/bin/opam',
                      '/usr/bin/onboard',
                      '/lib/init/upstart-job',
                      '/usr/bin/page',
                  ]),
                  'tag': lambda: random.choice(['<OLD>', '<RECENT-CTIME>', 'nan'])}


In [15]:
mock = pd.DataFrame(data=[get_mock_row(i) for i in range(len(popcon))],
                    columns=popcon.columns)
    


Upload the data

In [16]:
dataset = sy.Dataset(name="test", asset_list=[sy.Asset(name=f"weather", data=popcon,
                                                       mock=mock, mock_is_real=False)])
root_domain_client.upload_dataset(dataset)


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  6.03it/s]

100%|██████████| 1/1 [00:00<00:00,  5.99it/s]

Uploading: weather


<class 'syft.service.response.SyftSuccess'>: Dataset uploaded to 'pandas-test-domain-8'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [17]:
popcon.head()

,atime,ctime,package-name,mru-program,tag
0,1387295797,1367633260,perl-base,/usr/bin/perl,NaN
1,1387295796,1354370480,login,/bin/su,NaN
2,1387295743,1354341275,libtalloc2,/usr/lib/x86_64-linux-gnu/libtalloc.so.2.0.7,NaN
3,1387295743,1387224204,libwbclient0,/usr/lib/x86_64-linux-gnu/libwbclient.so.0,<RECENT-CTIME>
4,1387295742,1354341253,libselinux1,/lib/x86_64-linux-gnu/libselinux.so.1,NaN


## Create user account

In [18]:
user = root_domain_client.register(name="Jane Doe", email="jane@caltech.edu",
                            password="abc123", institution="Caltech", website="https://www.caltech.edu/")
# todo: give user data scientist role
guest_domain_client = node.client
guest_client = guest_domain_client.login(email="jane@caltech.edu", password="abc123")

Logged into pandas-test-domain-8 as <jane@caltech.edu>


# Data scientist: create syft_function

In [19]:
import numpy as np
import pandas as pd

## Summary

It's not obvious how to deal with Unix timestamps in pandas -- it took me quite a while to figure this out. The file we're using here is a popularity-contest file I found on my system at `/var/log/popularity-contest`.

## Get mocks

In [20]:
ds = guest_domain_client.datasets[0]

In [21]:
asset = ds.assets[0]

In [22]:
popcon = asset.mock

## Parsing Unix timestamps

The colums are the access time, created time, package name, recently used program, and a tag



In [23]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,135498594,135884828,libbsd0,/usr/bin/onboard,<RECENT-CTIME>
1,135156682,135373260,libbsd0,/lib/init/upstart-job,nan
2,135235843,135342428,libghc-stm-dev,/usr/bin/page,<RECENT-CTIME>
3,135156096,135124468,geoclue-ubuntu-geoip,/lib/init/upstart-job,<OLD>
4,135553112,135848504,geoclue-ubuntu-geoip,/lib/init/upstart-job,nan


The magical part about parsing timestamps in pandas is that numpy datetimes are already stored as Unix timestamps. So all we need to do is tell pandas that these integers are actually datetimes -- it doesn't need to do any conversion at all.

We need to convert these to ints to start:

In [24]:
popcon['atime'] = popcon['atime'].astype(int)
popcon['ctime'] = popcon['ctime'].astype(int)

Every numpy array and pandas series has a dtype -- this is usually `int64`, `float64`, or `object`. Some of the time types available are `datetime64[s]`, `datetime64[ms]`, and `datetime64[us]`. There are also `timedelta` types, similarly.

We can use the `pd.to_datetime` function to convert our integer timestamps into datetimes. This is a constant-time operation -- we're not actually changing any of the data, just how pandas thinks about it.

In [25]:
popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')

If we look at the dtype now, it's `<M8[ns]`. As far as I can tell `M8` is secret code for `datetime64`.

In [26]:
popcon['atime'].dtype

dtype('<M8[ns]')

So now we can look at our `atime` and `ctime` as dates!

In [27]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,1974-04-18 06:29:54,1974-04-22 17:47:08,libbsd0,/usr/bin/onboard,<RECENT-CTIME>
1,1974-04-14 07:31:22,1974-04-16 19:41:00,libbsd0,/lib/init/upstart-job,nan
2,1974-04-15 05:30:43,1974-04-16 11:07:08,libghc-stm-dev,/usr/bin/page,<RECENT-CTIME>
3,1974-04-14 07:21:36,1974-04-13 22:34:28,geoclue-ubuntu-geoip,/lib/init/upstart-job,<OLD>
4,1974-04-18 21:38:32,1974-04-22 07:41:44,geoclue-ubuntu-geoip,/lib/init/upstart-job,nan


Now suppose we want to look at all packages that aren't libraries.

First, I want to get rid of everything with timestamp 0. Notice how we can just use a string in this comparison, even though it's actually a timestamp on the inside? That is because pandas is amazing.


In [28]:
popcon = popcon[popcon['atime'] > '1970-01-01']

Now we can use pandas' magical string abilities to just look at rows where the package name doesn't contain 'lib'.

In [29]:
nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]

In [30]:
nonlibraries.sort_values('ctime', ascending=False)[:10]

,atime,ctime,package-name,mru-program,tag
1737,1974-04-17 14:30:32,1974-04-24 01:25:28,geoclue-ubuntu-geoip,/usr/bin/page,<OLD>
2409,1974-04-23 05:28:21,1974-04-24 01:18:23,ubuntu-extras-keyring,/usr/bin/page,nan
241,1974-04-17 09:34:32,1974-04-24 00:10:30,geoclue-ubuntu-geoip,/lib/init/upstart-job,<OLD>
1595,1974-04-24 00:36:31,1974-04-23 23:51:13,ubuntu-extras-keyring,/usr/bin/onboard,<OLD>
78,1974-04-23 08:45:04,1974-04-23 23:35:07,ubuntu-extras-keyring,/usr/bin/onboard,<RECENT-CTIME>
37,1974-04-22 03:16:40,1974-04-23 23:33:11,geoclue-ubuntu-geoip,/usr/bin/opam,<RECENT-CTIME>
543,1974-04-14 19:55:47,1974-04-23 23:19:37,ubuntu-extras-keyring,/usr/bin/opam,nan
2335,1974-04-23 12:34:00,1974-04-23 22:30:48,ubuntu-extras-keyring,/usr/bin/opam,nan
1486,1974-04-17 03:49:23,1974-04-23 22:17:16,geoclue-ubuntu-geoip,/usr/bin/onboard,<OLD>
1722,1974-04-19 21:14:53,1974-04-23 21:17:01,geoclue-ubuntu-geoip,/usr/bin/page,<RECENT-CTIME>


Okay, cool, it says that I I installed ddd recently. And postgresql! I remember installing those things. Neat.

The whole message here is that if you have a timestamp in seconds or milliseconds or nanoseconds, then you can just "cast" it to a `'datetime64[the-right-thing]'` and pandas/numpy will take care of the rest.

## Putting it together

Now we want to request the full code execution.

Let's put all that together, to prove how easy it is. 6 lines of magical pandas!

In [31]:
@sy.syft_function(input_policy=sy.ExactMatch(popcon=ds.assets[0]),
                  output_policy=sy.SingleExecutionExactOutput())
def find_recently_installed(popcon):
    import pandas as pd
    popcon['atime'] = popcon['atime'].astype(int)
    popcon['ctime'] = popcon['ctime'].astype(int)
    popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
    popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')
    popcon = popcon[popcon['atime'] > '1970-01-01']
    nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]
    return nonlibraries.sort_values('ctime', ascending=False)[:10]

Syft function 'find_recently_installed' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.


Create and submit project

In [32]:
new_project = sy.Project(
    name="Pandas Chapter 8",
    description="Hi, I would like to get some insights about the installed programs",
    members=[guest_domain_client],
)
new_project

syft.service.project.project.ProjectSubmit

In [33]:
project = new_project.start()
assert isinstance(project, sy.service.project.project.Project)
project

syft.service.project.project.Project

In [34]:
project.create_code_request(find_recently_installed, guest_domain_client)

<class 'syft.service.response.SyftSuccess'>: Code request for 'find_recently_installed' successfully added to 'Pandas Chapter 8' Project. To see code requests by a client, run `[your_client].code`

In [35]:
assert len(guest_domain_client.code.get_all())==1

In [36]:
assert len(project.events) == 1

In [37]:
assert isinstance(project.events[0], sy.service.project.project.ProjectRequest)

# Data owner: execute syft_function

In [38]:
from syft import MessageStatus

In [39]:
domain_client = node.login(email="info@openmined.org", password="changethis")

Logged into pandas-test-domain-8 as <info@openmined.org>


# Get messages

In [40]:
messages = domain_client.api.services.messages.get_all_unread()

In [41]:
messages

[syft.service.message.messages.Message, syft.service.message.messages.Message]

In [42]:
project_message = [x for x in messages if issubclass(x.linked_obj.object_type, Project)][0]

In [43]:
request = project_message.link.events[0].request
func = request.changes[0].link
op = func.output_policy_type

In [44]:
func

```python
class UserCode
    id: str = a11f0bdce6e44877a444b800920cd1f1
    status.approved: str = False
    service_func_name: str = find_recently_installed
    code:

@sy.syft_function(input_policy=sy.ExactMatch(popcon=ds.assets[0]),
                  output_policy=sy.SingleExecutionExactOutput())
def find_recently_installed(popcon):
    import pandas as pd
    popcon['atime'] = popcon['atime'].astype(int)
    popcon['ctime'] = popcon['ctime'].astype(int)
    popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
    popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')
    popcon = popcon[popcon['atime'] > '1970-01-01']
    nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]
    return nonlibraries.sort_values('ctime', ascending=False)[:10]

```

In [45]:
find_recently_installed = func.unsafe_function

In [46]:
real_data = domain_client.datasets[0].assets[0].data

In [47]:
real_data

,atime,ctime,package-name,mru-program,tag
0,1387295797,1367633260,perl-base,/usr/bin/perl,None
1,1387295796,1354370480,login,/bin/su,None
2,1387295743,1354341275,libtalloc2,/usr/lib/x86_64-linux-gnu/libtalloc.so.2.0.7,None
3,1387295743,1387224204,libwbclient0,/usr/lib/x86_64-linux-gnu/libwbclient.so.0,<RECENT-CTIME>
4,1387295742,1354341253,libselinux1,/lib/x86_64-linux-gnu/libselinux.so.1,None
...,...,...,...,...,...
2892,0,0,libreadline-dev,<NOFILES>,None
2893,0,0,notify-osd-icons,<NOFILES>,None
2894,0,0,python-apt-common,<NOFILES>,None
2895,0,0,libindicator-messages-status-provider1,<NOFILES>,None


In [48]:
real_result = find_recently_installed(popcon=real_data)

In [49]:
result = request.accept_by_depositing_result(real_result)
result
assert isinstance(result, sy.SyftSuccess)

# Data Owner: fetch result

In [50]:
guest_client.code[0].status

{NodeView(node_name='pandas-test-domain-8', verify_key=85af2a71bb544418c517e6ea96e4fb5805fe74c4181aad8924542360efff36d1): <UserCodeStatus.EXECUTE: 'execute'>}

In [51]:
asset = guest_client.datasets[0].assets[0]

In [52]:
result_ptr = guest_client.code.find_recently_installed(popcon=asset)
real_result = result_ptr.get()

In [53]:
real_result

,atime,ctime,package-name,mru-program,tag
57,2013-12-17 04:55:39,2013-12-17 04:55:42,ddd,/usr/bin/ddd,<RECENT-CTIME>
450,2013-12-16 20:03:20,2013-12-16 20:05:13,nodejs,/usr/bin/npm,<RECENT-CTIME>
454,2013-12-16 20:03:20,2013-12-16 20:05:04,switchboard-plug-keyboard,/usr/lib/plugs/pantheon/keyboard/options.txt,<RECENT-CTIME>
445,2013-12-16 20:03:20,2013-12-16 20:05:04,thunderbird-locale-en,/usr/lib/thunderbird-addons/extensions/langpac...,<RECENT-CTIME>
396,2013-12-16 20:08:27,2013-12-16 20:05:03,software-center,/usr/sbin/update-software-center,<RECENT-CTIME>
449,2013-12-16 20:03:20,2013-12-16 20:05:00,samba-common-bin,/usr/bin/net.samba3,<RECENT-CTIME>
397,2013-12-16 20:08:25,2013-12-16 20:04:59,postgresql-client-9.1,/usr/lib/postgresql/9.1/bin/psql,<RECENT-CTIME>
398,2013-12-16 20:08:23,2013-12-16 20:04:58,postgresql-9.1,/usr/lib/postgresql/9.1/bin/postmaster,<RECENT-CTIME>
452,2013-12-16 20:03:20,2013-12-16 20:04:55,php5-dev,/usr/include/php5/main/snprintf.h,<RECENT-CTIME>
440,2013-12-16 20:03:20,2013-12-16 20:04:54,php-pear,/usr/share/php/XML/Util.php,<RECENT-CTIME>


In [54]:
node.land()

Stopping pandas-test-domain-8
